<a href="https://colab.research.google.com/github/MutianWang/novel-cell/blob/main/human/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/Colab Notebooks/Brain Cell/data/'

## Preprocessing

In [ ]:
meta = pd.read_csv(path+'meta.csv', header=0)
cols_glut = meta[meta['class']=='Glutamatergic']['sample_name']
cols_non = meta[meta['class']=='Non-neuronal']['sample_name']
cols_gaba = meta[meta['class']=='GABAergic']['sample_name']

In [ ]:
def normalize(df):
    # counts per million
    df = df.div(df.sum(axis=1), axis=0) * 10**6
    df = df.fillna(0)
    return df

In [ ]:
def read_expression(file1, file2, usecols):
    # point-wise addition of exon and intron tables
    reader1 = pd.read_csv(file1, header=0, usecols=usecols, chunksize=1000)
    reader2 = pd.read_csv(file2, header=0, usecols=usecols, chunksize=1000)

    df = normalize(reader1.get_chunk() + reader2.get_chunk())
    for i in range(1, 51):
        df = pd.concat([df, normalize(reader1.get_chunk() + reader2.get_chunk())])
        if i%10==0:
            print('{}/50'.format(i))

    return df.transpose()

In [ ]:
exp_glut = read_expression(path+'exon.csv', path+'intron.csv', cols_glut) # 10525 * 50281
exp_non = read_expression(path+'exon.csv', path+'intron.csv', cols_non) # 914 * 50281
exp_gaba = read_expression(path+'exon.csv', path+'intron.csv', cols_gaba) # 4164 * 50281

10/50
20/50
30/50
40/50
50/50
10/50
20/50
30/50
40/50
50/50
10/50
20/50
30/50
40/50
50/50


In [ ]:
np.save(path+'exp_glut', exp_glut)
np.save(path+'exp_non', exp_non)
np.save(path+'exp_gaba', exp_gaba)

# clear RAM
del exp_glut, exp_non, exp_gaba

In [ ]:
exp_glut = np.load(path+'exp_glut.npy')[:5000-914]
exp_non = np.load(path+'exp_non.npy')

# first 4086 are Glutamatergic, last 914 are Non-neuronal
exp_train = np.concatenate([exp_glut, exp_non], axis=0)

# clear RAM
del exp_glut, exp_non

In [ ]:
np.save(path+'exp_train', exp_train)

## Dimension Reduction

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
pipe = Pipeline([('scaler1', StandardScaler()), ('pca', PCA(n_components=5000)), ('scaler2', MinMaxScaler())])

In [ ]:
#exp_train = np.load(path+'exp_train.npy')
exp_train = pipe.fit_transform(exp_train)
np.save(path+'exp_train_pca', exp_train)
del exp_train

In [ ]:
exp_test = np.load(path+'exp_gaba.npy')
exp_test = pipe.transform(exp_test)
np.save(path+'exp_test_pca', exp_test)
del exp_test

## GAN

In [4]:
exp_train = np.load(path+'exp_train_pca.npy')

In [5]:
dimension = exp_train.shape[1]

In [6]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [7]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=True, input_shape=(128,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512, use_bias=True))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512, use_bias=True))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(1024, use_bias=True))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(2048, use_bias=True))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # sigmoid function will make the range [0,1]
    model.add(layers.Dense(dimension, use_bias=True, activation='sigmoid'))

    return model

In [8]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(1024, use_bias=True, input_shape=(dimension,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(512, use_bias=True))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256, use_bias=True))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(1))

    return model

In [9]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [10]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2

In [11]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              5121024   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)              

In [12]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [13]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [14]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [15]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 128])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [16]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for data in dataset:
            train_step(data)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [17]:
BATCH_SIZE = 500
dataset = tf.data.Dataset.from_tensor_slices(exp_train).shuffle(1000).batch(BATCH_SIZE)

In [18]:
train(dataset, 10)

Time for epoch 1 is 3.2583796977996826 sec
Time for epoch 2 is 1.5587396621704102 sec
Time for epoch 3 is 1.5188868045806885 sec
Time for epoch 4 is 1.5487749576568604 sec
Time for epoch 5 is 1.4790289402008057 sec
Time for epoch 6 is 1.4789140224456787 sec
Time for epoch 7 is 1.4853599071502686 sec
Time for epoch 8 is 1.4999761581420898 sec
Time for epoch 9 is 1.475475788116455 sec
Time for epoch 10 is 1.4388682842254639 sec


In [19]:
exp_gen = generator(tf.random.normal([1000,128]))

## Evaluation

### mean center

In [20]:
from scipy.spatial import distance

In [21]:
mean_glut = np.mean(np.load(path+'exp_train_pca.npy')[:4086], axis=0)
mean_non = np.mean(np.load(path+'exp_train_pca.npy')[4086:], axis=0)
mean_gaba = np.mean(np.load(path+'exp_test_pca.npy'), axis=0)

In [49]:
# last value of mean_gaba is too small, so it's excluded when computing the distance
# this approach is identical to reducing the dimension to 4999
mean_gaba[-1]

-1.597220239451883e+22

In [34]:
# L1 distance
res = [0, 0, 0]
tmp = []
for exp in exp_gen:
    d1 = np.sum(np.abs(exp-mean_glut)[:-1])
    d2 = np.sum(np.abs(exp-mean_non)[:-1])
    d3 = np.sum(np.abs(exp-mean_gaba)[:-1])
    tmp.append(d2)
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

[734, 266, 0]


In [48]:
# L2 distance
res = [0, 0, 0]
for i, exp in enumerate(exp_gen):
    d1 = np.sum(np.square(exp-mean_glut)[:-1])
    d2 = np.sum(np.square(exp-mean_non)[:-1])
    d3 = np.sum(np.square(exp-mean_gaba)[:-1])
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in square
  


[733, 267, 0]


In [53]:
# cosine distance
res = [0, 0, 0]
for exp in exp_gen:
    d1 = distance.cosine(exp[:-1], mean_glut[:-1])
    d2 = distance.cosine(exp[:-1], mean_non[:-1])
    d3 = distance.cosine(exp[:-1], mean_gaba[:-1])
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

[748, 252, 0]


### median center

In [54]:
median_glut = np.median(np.load(path+'exp_train_pca.npy')[:4086], axis=0)
median_non = np.median(np.load(path+'exp_train_pca.npy')[4086:], axis=0)
median_gaba = np.median(np.load(path+'exp_test_pca.npy'), axis=0)

In [62]:
# last value of mean_gaba is too small, so it's excluded when computing the distance
median_gaba[-1]

-2.923588706429223e+21

In [63]:
# L1 distance
res = [0, 0, 0]
tmp = []
for exp in exp_gen:
    d1 = np.sum(np.abs(exp-median_glut)[:-1])
    d2 = np.sum(np.abs(exp-median_non)[:-1])
    d3 = np.sum(np.abs(exp-median_gaba)[:-1])
    tmp.append(d2)
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

[731, 269, 0]


In [64]:
# L2 distance
res = [0, 0, 0]
for i, exp in enumerate(exp_gen):
    d1 = np.sum(np.square(exp-median_glut)[:-1])
    d2 = np.sum(np.square(exp-median_non)[:-1])
    d3 = np.sum(np.square(exp-median_gaba)[:-1])
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in square
  


[731, 269, 0]


In [65]:
# cosine distance
res = [0, 0, 0]
for exp in exp_gen:
    d1 = distance.cosine(exp[:-1], median_glut[:-1])
    d2 = distance.cosine(exp[:-1], median_non[:-1])
    d3 = distance.cosine(exp[:-1], median_gaba[:-1])
    i = np.argmin([d1, d2, d3])
    res[i] += 1

assert sum(res) == exp_gen.shape[0]

print(res)

[740, 257, 3]
